In [36]:
from pytorch_pretrained import BertModel, BertTokenizer

In [37]:
bert = BertModel.from_pretrained('./bert_pretrain/roberta')
tokenizer = BertTokenizer.from_pretrained('./bert_pretrain/roberta')

In [3]:
import torch
torch.cuda.empty_cache()
bert.cuda()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (

In [52]:
for name, p in bert.named_parameters():
    print(name)

embeddings.word_embeddings.weight
embeddings.position_embeddings.weight
embeddings.token_type_embeddings.weight
embeddings.LayerNorm.weight
embeddings.LayerNorm.bias
encoder.layer.0.attention.self.query.weight
encoder.layer.0.attention.self.query.bias
encoder.layer.0.attention.self.key.weight
encoder.layer.0.attention.self.key.bias
encoder.layer.0.attention.self.value.weight
encoder.layer.0.attention.self.value.bias
encoder.layer.0.attention.output.dense.weight
encoder.layer.0.attention.output.dense.bias
encoder.layer.0.attention.output.LayerNorm.weight
encoder.layer.0.attention.output.LayerNorm.bias
encoder.layer.0.intermediate.dense.weight
encoder.layer.0.intermediate.dense.bias
encoder.layer.0.output.dense.weight
encoder.layer.0.output.dense.bias
encoder.layer.0.output.LayerNorm.weight
encoder.layer.0.output.LayerNorm.bias
encoder.layer.1.attention.self.query.weight
encoder.layer.1.attention.self.query.bias
encoder.layer.1.attention.self.key.weight
encoder.layer.1.attention.self.key

In [5]:
tokenizer

In [6]:
juzi = [['[CLS]', '我','们','今','天','晚','上','吃','什','么'],['[CLS]', '我','们','今','天','晚','上','吃','什','么']]
juzi_to_id1 = tokenizer.convert_tokens_to_ids(juzi[0])
juzi_to_id2 = tokenizer.convert_tokens_to_ids(juzi[1])
juzi_to_id = [juzi_to_id1, juzi_to_id2]
device = torch.device('cuda:0')

In [7]:
import torch
juzi_to_id = torch.tensor(juzi_to_id).long().to(device)
mask = [[1,1,1,1,1,1,1,1,1,1],[1,1,1,1,1,1,1,1,1,1]]
mask = torch.tensor(mask).long().to(device)
print(juzi_to_id)
print(mask)

tensor([[ 101, 2769,  812,  791, 1921, 3241,  677, 1391,  784,  720],
        [ 101, 2769,  812,  791, 1921, 3241,  677, 1391,  784,  720]],
       device='cuda:0')
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')


In [8]:
hidde, pooled = bert(juzi_to_id, attention_mask=mask, output_all_encoded_layers=False)

In [43]:
pooled  # 2, 1024

tensor([[-0.8292, -0.1648, -0.2692,  ...,  0.3706, -0.4522,  0.3950],
        [-0.6834, -0.3769, -0.1528,  ...,  0.3388, -0.5243,  0.1920]],
       device='cuda:0', grad_fn=<TanhBackward>)

In [10]:
hidde  # 2， 10， 1024

tensor([[[ 0.2666,  0.6644,  0.2532,  ..., -0.7685,  0.2291, -0.2850],
         [ 0.0529,  0.2937, -1.5961,  ..., -0.2821, -0.1782,  0.2549],
         [ 0.0061,  1.4136, -1.1018,  ..., -0.9215, -0.4790,  0.5188],
         ...,
         [ 0.8160,  0.2423, -0.9496,  ..., -0.3439,  0.4828,  0.8609],
         [ 1.2142,  0.0394,  0.2263,  ..., -0.8174, -0.1919,  0.8640],
         [ 1.9839,  0.7616, -0.0596,  ..., -0.6130, -0.1420,  0.4426]],

        [[ 0.0076,  0.9252, -0.1191,  ..., -0.5722,  0.4236,  0.1524],
         [-0.6485,  0.6202, -1.8303,  ..., -0.8842, -0.0305,  0.4227],
         [-0.2697,  1.6067, -1.1103,  ..., -0.7198, -0.4453,  0.6600],
         ...,
         [ 0.5594,  0.2290, -1.0874,  ..., -0.4127,  1.0300,  1.1115],
         [ 1.0699,  0.5532,  0.1946,  ..., -1.0279,  0.2828,  0.5622],
         [ 0.9150,  1.0474, -0.5856,  ...,  0.6641, -0.0062,  1.1352]]],
       device='cuda:0', grad_fn=<AddBackward0>)

In [44]:
import torch.nn as nn
fc = nn.Linear(1024, 10).cuda()
out = fc(pooled)

In [45]:
out

tensor([[ 0.1798, -0.3242, -0.4176,  0.1894, -0.0802,  0.3324,  0.0392, -0.2806,
         -0.2077,  0.1011],
        [ 0.2392, -0.4074, -0.3829,  0.1737,  0.0490,  0.3623, -0.1258, -0.3852,
         -0.0295,  0.1514]], device='cuda:0', grad_fn=<AddmmBackward>)

In [46]:
out.shape

torch.Size([2, 10])

In [48]:
from pytorch_pretrained import BertModel, BertTokenizer
import torch
import torch.nn as nn

# 加载本地预训练模型文件，包括config.json, vocab.txt, pytorch_model.bin
bert = BertModel.from_pretrained('./bert_pretrain/roberta')  
tokenizer = BertTokenizer.from_pretrained('./bert_pretrain/roberta')

bert.cuda()    # 将bert放到cuda上
fc = nn.Linear(1024, 10).cuda()   # 10为类别
device = torch.device('cuda:0')

juzi = [['[CLS]', '我','们','今','天','晚','上','吃','什','么'],
		['[CLS]', '今','天','的','天','气','真','的','好','呀']]
juzi_to_id1 = tokenizer.convert_tokens_to_ids(juzi[0])
juzi_to_id2 = tokenizer.convert_tokens_to_ids(juzi[1])
juzi_to_id = [juzi_to_id1, juzi_to_id2]
juzi_to_id = torch.tensor(juzi_to_id).long().to(device)
mask = [[1,1,1,1,1,1,1,1,1,1],[1,1,1,1,1,1,1,1,1,1]]
mask = torch.tensor(mask).long().to(device)
# print(juzi_to_id)
# print(mask)

encoder_layer, pooled = bert(juzi_to_id, attention_mask=mask,
output_all_encoded_layers=False)

print(encoder_layer.shape)  # 2, 10, 768
print(pooled.shape)   # 2, 768
out = fc(pooled)      # 2, 10

RuntimeError: CUDA out of memory. Tried to allocate 84.00 MiB (GPU 0; 2.00 GiB total capacity; 1.27 GiB already allocated; 7.00 MiB free; 1.28 GiB reserved in total by PyTorch)